In [2]:
import numpy as np 
import pandas as pd 
from catboost import CatBoostRegressor
from tqdm import tqdm
import gc
import datetime as dt
pd.options.display.max_columns = 999

In [10]:

print('Loading Properties ...')
prop = pd.read_csv("final_prop_cat.csv", low_memory = False)
print('Loading Train ...')
train = pd.read_csv("train.csv", parse_dates=['transactiondate'], low_memory=False)
print('Loading Done')

Loading Properties ...
Loading Train ...
Loading Done


In [11]:
prop.head()

,parcelid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,assessmentyear,censustractandblock
0,10754147,-999.0,0.0,0.0,-999.0,5.0,-999.0,3652.0,3652.0,-999.0,-999.0,6037.0,-999.0,5.0,-999.0,-999.0,-999.0,34144442.0,-118654084.0,85768.0,010D,269.0,6.037800e+07,37688.0,3101.0,-999.0,96337.0,0.0,-999.0,-999.0,1987.0,-999.0,2016.0,-2.147484e+09
1,10759547,-999.0,0.0,0.0,-999.0,1.0,-999.0,800.0,800.0,-999.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,-999.0,34140430.0,-118625364.0,4083.0,0109,261.0,6.037800e+07,37688.0,3101.0,-999.0,96337.0,0.0,-999.0,-999.0,1938.0,-999.0,2015.0,-2.147484e+09
2,10843547,-999.0,0.0,0.0,-999.0,1.0,-999.0,73026.0,1274.0,73026.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,-999.0,33989359.0,-118394633.0,63085.0,1200,47.0,6.037703e+07,51617.0,3101.0,-999.0,96095.0,0.0,-999.0,2.0,1959.0,1.0,2016.0,-2.147484e+09
3,10859147,-999.0,0.0,0.0,6.0,2.0,-999.0,5068.0,1041.0,5068.0,-999.0,6037.0,-999.0,2.0,-999.0,-999.0,-999.0,34148863.0,-118437206.0,7521.0,1200,47.0,6.037141e+07,12447.0,3101.0,27080.0,96424.0,0.0,-999.0,-999.0,1948.0,1.0,2016.0,-2.147484e+09
4,10879947,-999.0,0.0,0.0,-999.0,1.0,-999.0,1776.0,784.0,1776.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,-999.0,34194168.0,-118385816.0,8512.0,1210,31.0,6.037123e+07,12447.0,3101.0,46795.0,96450.0,0.0,-999.0,1.0,1947.0,1.0,2016.0,-2.147484e+09


In [13]:
train.head()

,parcelid,logerror,transaction_year,transaction_month,transaction_day,transaction_quarter
0,14170803,-0.032500,2016,6,22,2
1,11457802,0.050663,2017,19,7,7
2,12762961,-0.138100,2016,1,14,1
3,11634161,-0.046000,2016,8,1,3
4,10748445,0.023700,2016,5,9,2


In [14]:
train.tail()

,parcelid,logerror,transaction_year,transaction_month,transaction_day,transaction_quarter
109122,10890361,0.574225,2017,20,17,7
109123,17095451,-0.002108,2017,15,22,5
109124,13959346,-0.006000,2016,7,18,3
109125,17139623,-0.022538,2017,19,5,7
109126,14482715,0.019304,2017,17,19,6


In [12]:
def add_date_features(df):
    df["transaction_year"] = df["transactiondate"].dt.year
    df["transaction_month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month
    df["transaction_day"] = df["transactiondate"].dt.day
    df["transaction_quarter"] = (df["transactiondate"].dt.year - 2016)*4 +df["transactiondate"].dt.quarter
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df
train = add_date_features(train)

In [15]:
sample_submission  = pd.read_csv("new_submission.csv", low_memory=False).iloc[:,1:]
sample_submission.columns = ['parcelid','201704','201705','201706','201707','201708','201709']

In [16]:
prop.head()

,parcelid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,assessmentyear,censustractandblock
0,10754147,-999.0,0.0,0.0,-999.0,5.0,-999.0,3652.0,3652.0,-999.0,-999.0,6037.0,-999.0,5.0,-999.0,-999.0,-999.0,34144442.0,-118654084.0,85768.0,010D,269.0,6.037800e+07,37688.0,3101.0,-999.0,96337.0,0.0,-999.0,-999.0,1987.0,-999.0,2016.0,-2.147484e+09
1,10759547,-999.0,0.0,0.0,-999.0,1.0,-999.0,800.0,800.0,-999.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,-999.0,34140430.0,-118625364.0,4083.0,0109,261.0,6.037800e+07,37688.0,3101.0,-999.0,96337.0,0.0,-999.0,-999.0,1938.0,-999.0,2015.0,-2.147484e+09
2,10843547,-999.0,0.0,0.0,-999.0,1.0,-999.0,73026.0,1274.0,73026.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,-999.0,33989359.0,-118394633.0,63085.0,1200,47.0,6.037703e+07,51617.0,3101.0,-999.0,96095.0,0.0,-999.0,2.0,1959.0,1.0,2016.0,-2.147484e+09
3,10859147,-999.0,0.0,0.0,6.0,2.0,-999.0,5068.0,1041.0,5068.0,-999.0,6037.0,-999.0,2.0,-999.0,-999.0,-999.0,34148863.0,-118437206.0,7521.0,1200,47.0,6.037141e+07,12447.0,3101.0,27080.0,96424.0,0.0,-999.0,-999.0,1948.0,1.0,2016.0,-2.147484e+09
4,10879947,-999.0,0.0,0.0,-999.0,1.0,-999.0,1776.0,784.0,1776.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,-999.0,34194168.0,-118385816.0,8512.0,1210,31.0,6.037123e+07,12447.0,3101.0,46795.0,96450.0,0.0,-999.0,1.0,1947.0,1.0,2016.0,-2.147484e+09


In [17]:
train = pd.merge(train, prop, how='left' ,on = 'parcelid')

In [ ]:
train.iloc[:, train.columns.str.startswith('tax')] = np.nan

In [18]:
train_df = train
test_df = pd.merge(sample_submission[['parcelid']], prop, how='left',on='parcelid')

In [19]:
del prop, train
gc.collect();

In [20]:
print('Remove missing data fields ...')

missing_perc_thresh = 0.98
exclude_missing = []
num_rows = train_df.shape[0]
for c in train_df.columns:
    num_missing = train_df[c].isnull().sum()
    if num_missing == 0:
        continue
    missing_frac = num_missing / float(num_rows)
    if missing_frac > missing_perc_thresh:
        exclude_missing.append(c)
print("We exclude: %s" % len(exclude_missing))

Remove missing data fields ...
We exclude: 0


In [21]:
del num_rows, missing_perc_thresh
gc.collect();

In [22]:
print ("Remove features with one unique value !!")
exclude_unique = []
for c in train_df.columns:
    num_uniques = len(train_df[c].unique())
    if train_df[c].isnull().sum() != 0:
        num_uniques -= 1
    if num_uniques == 1:
        exclude_unique.append(c)
print("We exclude: %s" % len(exclude_unique))

Remove features with one unique value !!
We exclude: 1


In [23]:
train_df.columns

Index(['parcelid', 'logerror', 'transaction_year', 'transaction_month',
       'transaction_day', 'transaction_quarter', 'airconditioningtypeid',
       'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet15', 'finishedsquarefeet50', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'propertylandusetypeid',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
       'unitcnt', 'yearbuilt', 'numberofstories', 'assessmentyear',
       'censustractandblock'],
      dtype='object')

In [24]:
print ("Define training features !!")
exclude_other = ['parcelid', 'logerror','subdate']
train_features = []
for c in train_df.columns:
    if c not in exclude_other:
        train_features.append(c)
print("We use these for training: %s" % len(train_features))

Define training features !!
We use these for training: 37


In [25]:
print ("Define categorial features !!")
cat_feature_inds = []
cat_unique_thresh = 1000
for i, c in enumerate(train_features):
    num_uniques = len(train_df[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)

Define categorial features !!


In [26]:
cat_feature_inds

[0, 1, 2, 3, 4, 7, 14, 19, 23, 24, 26, 27, 28, 29, 33, 35]

In [27]:
print("Cat features are: %s" % [train_features[ind] for ind in cat_feature_inds])

Cat features are: ['transaction_year', 'transaction_month', 'transaction_day', 'transaction_quarter', 'airconditioningtypeid', 'buildingqualitytypeid', 'fips', 'heatingorsystemtypeid', 'propertycountylandusecode', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'yearbuilt', 'assessmentyear']


In [28]:
print ("Replacing NaN values by -999 !!")
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

Replacing NaN values by -999 !!


In [46]:
print ("Training time !!")
X_train = train_df[train_features]
y_train = train_df.logerror
print(X_train.shape, y_train.shape)

Training time !!
(109127, 37) (109127,)


In [30]:
test_df['transactiondate'] = pd.Timestamp('2016-12-01') 
test_df = add_date_features(test_df)
X_test = test_df[train_features]
print(X_test.shape)

(2985217, 37)


In [47]:
X_train.head()

,transaction_year,transaction_month,transaction_day,transaction_quarter,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,assessmentyear,censustractandblock
0,2016,6,22,2,-999.0,3.0,4.0,-999.0,3.0,-999.0,1966.0,1966.0,-999.0,-999.0,6059.0,1.0,3.0,2.0,400.0,-999.0,33897045.0,-117867375.0,7000.0,122,261.0,6.059012e+07,26531.0,1286.0,-999.0,97068.0,6.0,-999.0,-999.0,2010.0,1.0,2016.0,6.059012e+13
1,2017,19,7,7,-999.0,1.0,2.0,6.0,1.0,-999.0,1193.0,1193.0,-999.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,7.0,33960844.0,-118391885.0,6000.0,0100,261.0,6.037276e+07,12447.0,3101.0,7877.0,96026.0,0.0,-999.0,1.0,1941.0,-999.0,2016.0,6.037276e+13
2,2016,1,14,1,-999.0,2.0,4.0,6.0,2.0,-999.0,1591.0,1591.0,-999.0,-999.0,6037.0,-999.0,2.0,-999.0,-999.0,2.0,33892219.0,-118009713.0,6079.0,0100,261.0,6.037504e+07,5465.0,3101.0,-999.0,96190.0,0.0,-999.0,1.0,1955.0,-999.0,2016.0,6.037504e+13
3,2016,8,1,3,-999.0,1.0,2.0,4.0,1.0,-999.0,784.0,784.0,-999.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,-999.0,34087724.0,-118383207.0,2015.0,0100,261.0,6.037701e+07,48424.0,3101.0,-999.0,96050.0,0.0,-999.0,1.0,1930.0,-999.0,2016.0,6.037701e+13
4,2016,5,9,2,1.0,2.0,2.0,7.0,2.0,-999.0,2545.0,2545.0,-999.0,-999.0,6037.0,-999.0,2.0,-999.0,-999.0,2.0,34133621.0,-118821788.0,104201.0,010C,266.0,6.037800e+07,41673.0,3101.0,-999.0,96384.0,0.0,-999.0,1.0,1996.0,-999.0,2016.0,6.037800e+13


In [48]:
y_train.head()

0   -0.032500
1    0.050663
2   -0.138100
3   -0.046000
4    0.023700
Name: logerror, dtype: float64

In [31]:
num_ensembles = 5
y_pred = 0.0
for i in tqdm(range(num_ensembles)):
    model = CatBoostRegressor(
        iterations=900, learning_rate=0.02,
        depth=7, l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=i*i)
    model.fit(
        X_train, y_train,
        cat_features=cat_feature_inds)
    y_pred += model.predict(X_test)
y_pred /= num_ensembles

100%|█████████████████████████████████████████| 5/5 [1:05:05<00:00, 781.01s/it]


In [32]:
submission = pd.DataFrame({
    'parcelid': test_df['parcelid'],
})

In [33]:
test_dates = {
    '201704': pd.Timestamp('2016-04-30'),
    '201705': pd.Timestamp('2016-05-31'),
    '201706': pd.Timestamp('2016-06-30'),
    '201707': pd.Timestamp('2017-07-30'),
    '201708': pd.Timestamp('2017-08-31'),
    '201709': pd.Timestamp('2017-09-30')
}

In [34]:
test_dates

{'201704': Timestamp('2016-04-30 00:00:00'),
 '201705': Timestamp('2016-05-31 00:00:00'),
 '201706': Timestamp('2016-06-30 00:00:00'),
 '201707': Timestamp('2017-07-30 00:00:00'),
 '201708': Timestamp('2017-08-31 00:00:00'),
 '201709': Timestamp('2017-09-30 00:00:00')}

In [35]:
for label, test_date in test_dates.items():
    print("Predicting for: %s ... " % (label))
    submission[label] = y_pred

Predicting for: 201704 ... 
Predicting for: 201705 ... 
Predicting for: 201706 ... 
Predicting for: 201707 ... 
Predicting for: 201708 ... 
Predicting for: 201709 ... 


In [38]:
submission.to_csv('Cat_Boost_Our_Cat_Prop.csv', float_format='%.6f',index=False)

In [39]:
sample_submission  = pd.read_csv("new_submission.csv", low_memory=False).iloc[:,1:]

In [41]:
sample_submission.head()

,parcelid,201704,201705,201706,201707,201708,201709
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [42]:
test_path = "C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\test.csv"
test = pd.read_csv(test_path, low_memory=False)
test_size = len(test)

In [43]:
test.head()

,parcelid,logerror,transactiondate,subdate
0,12255126,-0.010475,1/4/2017,201704
1,11699092,-0.135031,1/4/2017,201704
2,11135506,0.065301,1/4/2017,201704
3,11045924,0.004267,1/4/2017,201704
4,17149086,0.075498,1/4/2017,201704


In [44]:
submission.head()

,parcelid,201704,201705,201706,201707,201708,201709
0,10754147,0.013667,0.013667,0.013667,0.013667,0.013667,0.013667
1,10759547,0.007301,0.007301,0.007301,0.007301,0.007301,0.007301
2,10843547,0.038225,0.038225,0.038225,0.038225,0.038225,0.038225
3,10859147,0.046890,0.046890,0.046890,0.046890,0.046890,0.046890
4,10879947,0.024258,0.024258,0.024258,0.024258,0.024258,0.024258


In [45]:
from math import fabs as absolute
total_ae = 0.0
print("Processing ", test_size, "rows")
for index,row in test.iterrows():
    r = submission.loc[submission['parcelid'] == row[0]]
    
    pred_error = r[str(row[3])]
    total_ae += absolute(pred_error - row[1])
    
    if index % 10000 == 0 and index !=0:
        print("Completed: ", index, "rows")
        
print("===Process Completed===")
print("\n Final score:", total_ae/test_size)

Processing  54849 rows
Completed:  10000 rows
Completed:  20000 rows
Completed:  30000 rows
Completed:  40000 rows
Completed:  50000 rows
===Process Completed===

 Final score: 0.0682537600631674


In [146]:
submission.to_csv('cat_final.csv')

In [49]:
X_train.head()

,transaction_year,transaction_month,transaction_day,transaction_quarter,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,assessmentyear,censustractandblock
0,2016,6,22,2,-999.0,3.0,4.0,-999.0,3.0,-999.0,1966.0,1966.0,-999.0,-999.0,6059.0,1.0,3.0,2.0,400.0,-999.0,33897045.0,-117867375.0,7000.0,122,261.0,6.059012e+07,26531.0,1286.0,-999.0,97068.0,6.0,-999.0,-999.0,2010.0,1.0,2016.0,6.059012e+13
1,2017,19,7,7,-999.0,1.0,2.0,6.0,1.0,-999.0,1193.0,1193.0,-999.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,7.0,33960844.0,-118391885.0,6000.0,0100,261.0,6.037276e+07,12447.0,3101.0,7877.0,96026.0,0.0,-999.0,1.0,1941.0,-999.0,2016.0,6.037276e+13
2,2016,1,14,1,-999.0,2.0,4.0,6.0,2.0,-999.0,1591.0,1591.0,-999.0,-999.0,6037.0,-999.0,2.0,-999.0,-999.0,2.0,33892219.0,-118009713.0,6079.0,0100,261.0,6.037504e+07,5465.0,3101.0,-999.0,96190.0,0.0,-999.0,1.0,1955.0,-999.0,2016.0,6.037504e+13
3,2016,8,1,3,-999.0,1.0,2.0,4.0,1.0,-999.0,784.0,784.0,-999.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,-999.0,34087724.0,-118383207.0,2015.0,0100,261.0,6.037701e+07,48424.0,3101.0,-999.0,96050.0,0.0,-999.0,1.0,1930.0,-999.0,2016.0,6.037701e+13
4,2016,5,9,2,1.0,2.0,2.0,7.0,2.0,-999.0,2545.0,2545.0,-999.0,-999.0,6037.0,-999.0,2.0,-999.0,-999.0,2.0,34133621.0,-118821788.0,104201.0,010C,266.0,6.037800e+07,41673.0,3101.0,-999.0,96384.0,0.0,-999.0,1.0,1996.0,-999.0,2016.0,6.037800e+13


In [50]:
y_train.head()

0   -0.032500
1    0.050663
2   -0.138100
3   -0.046000
4    0.023700
Name: logerror, dtype: float64

In [56]:

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import random
import datetime as dt
from sklearn.metrics import mean_absolute_error

In [57]:
y_mean = np.median(y_train)

In [58]:
print("\nSetting up data for XGBoost ...")
# xgboost params
xgb_params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,   
    'alpha': 0.4, 
    'base_score': y_mean,
    'silent': 1
}
num_boost_rounds = 500


Setting up data for XGBoost ...


In [59]:
X_train.shape

(109127, 37)

In [60]:
x_train = X_train.iloc[:65000,:]

In [61]:
x_train.tail()

,transaction_year,transaction_month,transaction_day,transaction_quarter,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,assessmentyear,censustractandblock
64995,2017,17,11,6,-999.0,1.0,2.0,5.0,1.0,-999.0,1120.0,1120.0,-999.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,7.0,33999024.0,-118256798.0,5001.0,0100,261.0,6.037229e+07,12447.0,3101.0,268496.0,95992.0,0.0,-999.0,1.0,1908.0,-999.0,2016.0,6.037229e+13
64996,2017,15,29,5,1.0,5.0,5.0,7.0,5.0,-999.0,3421.0,3421.0,-999.0,-999.0,6037.0,-999.0,5.0,-999.0,-999.0,2.0,34684553.0,-118217216.0,6588.0,0100,261.0,6.037901e+07,5534.0,3101.0,-999.0,97319.0,0.0,-999.0,1.0,2006.0,-999.0,2016.0,6.037901e+13
64997,2017,18,28,6,-999.0,2.0,2.0,-999.0,2.0,-999.0,1020.0,1020.0,-999.0,-999.0,6059.0,-999.0,2.0,-999.0,-999.0,-999.0,33771600.0,-117979000.0,2560.0,34,266.0,6.059089e+07,24832.0,1286.0,-999.0,97051.0,0.0,-999.0,-999.0,1987.0,-999.0,2016.0,6.059089e+13
64998,2016,4,21,2,1.0,5.0,5.0,9.0,5.0,-999.0,3653.0,3653.0,-999.0,-999.0,6037.0,-999.0,5.0,-999.0,-999.0,2.0,34160952.0,-118452824.0,6749.0,0101,261.0,6.037129e+07,12447.0,3101.0,27080.0,96412.0,0.0,-999.0,1.0,2006.0,-999.0,2016.0,6.037129e+13
64999,2016,10,14,4,-999.0,1.0,3.0,4.0,1.0,-999.0,919.0,919.0,-999.0,-999.0,6037.0,-999.0,1.0,-999.0,-999.0,7.0,34020728.0,-118203453.0,4200.0,0100,261.0,6.037205e+07,12447.0,3101.0,113886.0,96004.0,0.0,-999.0,1.0,1925.0,-999.0,2016.0,6.037205e+13


In [62]:
x_val = X_train.iloc[65000:,:]

In [63]:
target = y_train

In [65]:
y_train = target.iloc[:65000]
y_val = target.iloc[65000:]

In [67]:
print('Shape train: {}\nShape validation: {}'.format(x_train.shape, x_val.shape))

Shape train: (65000, 37)
Shape validation: (44127, 37)


In [68]:
y_mean = np.median(y_train)